In [1]:
!pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/7c/2e/60e624bbe16f4baa45cb6e48a1ee05edd48a0a14cceec4d7eec9258755ac/streamlit-1.38.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/9b/52/4a86a4fa1cc2aae79137cc9510b7080c3e5aede2310d14fae5486feec7f7/altair-5.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a4/07/14f8ad37f2d12a5ce41206c21820d8cb6561b728e51fad4530dff0552a67/cachetools-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydeck<1,>=0.8.0b4 from https://files.pythonhosted.org/packages/ab/4c/b888e6cf58bd9db9c93f40d1c6be8283ff49d88919231afe93a6bcf61626/pydeck-0.9.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for narwhals>=1.5.2 from https://files.pythonhosted.org/packages/c1/1a/6d69e6b677d2caad40b8315429010f9a0677b40fd5aa071cbe6ea0e

In [5]:
import streamlit as st
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
import numpy as np
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from mtcnn import MTCNN
import os
import pickle
import tempfile



# Initialize the face detector and feature extractor
detector = MTCNN()
model = resnet50(pretrained=True)
model.fc = nn.Identity()  # For feature extraction
model.eval()

# Preprocessing pipeline
preprocess = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to extract face embeddings for the user image
def load_face_embeddings():
    # Load the precomputed celebrity embeddings from a .pkl file
    with open('celeb_embeddings.pkl', 'rb') as f:
        celeb_embeddings = pickle.load(f)
    
    return celeb_embeddings

# Compare user image with precomputed celebrity embeddings
def find_celebrity_look_alike(user_embedding, celeb_embeddings, top_n=3):
    similarities = []
    
    # Compare the user embedding with each celebrity embedding
    for celeb_name, celeb_embedding in celeb_embeddings.items():
        similarity = cosine_similarity(user_embedding, celeb_embedding)[0][0]
        similarities.append((celeb_name, similarity))
    
    # Sort by similarity (highest first)
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    
    # Return the top N most similar celebrities
    return similarities[:top_n] if similarities else "No face detected in any celebrity images."

# Streamlit app
def main():
    st.title("Celebrity Look-Alike Finder")
    st.write("Upload a photo and find your celebrity look-alike!")
    celeb_embeddings=load_face_embeddings()
    # File upload for user image
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    
    if uploaded_file is not None:
        # Convert the uploaded file to an opencv image
        img = Image.open(uploaded_file)
        img = np.array(img)
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg")
        cv2.imwrite(temp_file.name, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        
        # Display the uploaded image
        st.image(img, caption="Uploaded Image", use_column_width=True)
        st.write("")

        # Run the model and find top celebrity look-alikes
        if st.button("Find Look-Alikes"):
            st.write("Processing the image...")
            
            user_embedding = extract_face_embedding(temp_file.name)
            if user_embedding is not None:
                top_matches = find_celebrity_look_alike(user_embedding, celeb_embeddings)
                
                # Display results
                st.write(f"Top {len(top_matches)} Celebrity Matches:")
                for i, (celeb_name, similarity) in enumerate(top_matches):
                    st.write(f"Match {i+1}: {celeb_name} with similarity score of {similarity}")
                    celeb_image_path = os.path.join('small_celeba', celeb_name)
                    celeb_image = Image.open(celeb_image_path)
                    st.image(celeb_image, caption=f"Celebrity {i+1}", use_column_width=True)
            else:
                st.write("No face detected in the user image.")

if __name__ == "__main__":
    main()


2024-09-06 13:11:45.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:11:45.604 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:11:45.604 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:11:45.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:11:45.606 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:11:45.606 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:11:45.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:11:45.612 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar